In [1]:
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from lowAltitude_classification.gsd_utils import papermode

In [2]:
font_size = 8
papermode(plt, font_size, has_latex=True)
rc('text.latex', preamble=r'\usepackage{color}')
matplotlib.use('ps')

In [3]:
# Load the data
data = pd.read_csv('results/phase3-test-M2F_METRICS_new.csv')
data

,PL_Version,experiment,F1,pAcc
0,NaN,SUPERVISED,0.2683,0.3453
1,V1,PRETRAIN,0.4382,0.5550
2,NaN,MovingWINDOW,0.4069,0.5262
3,V1,FINETUNE,0.4556,0.5970


In [6]:
from matplotlib.transforms import Bbox

results_f1 = {
    r'Supervised' "\n"r'(\texttt{FT})': {'f1': data.iloc[0]['F1'], 'use_drones': True, 'use_pl': False, 'img': 'annotations/1.png'},
    r'Pseudo-labels'"\n"'(with voting)': {'f1': data.iloc[2]['F1'], 'use_drones': False, 'use_pl': False, 'img': 'voting/1.png'},
    r'Pretrained'"\n"r'(\texttt{PT})': {'f1': data.iloc[1]['F1'], 'use_drones': False, 'use_pl': True, 'img': 'pt/1.png'},
    r'Finetuned'"\n"r'(\texttt{PT+FT})': {'f1': data.iloc[3]['F1'], 'use_drones': True, 'use_pl': True, 'img': 'pt_ft/1.png'},
}

# results_f1 = {k: round(v, 4) * 100 for k, v in results_f1.items()}
for k, v in results_f1.items():
    results_f1[k]['f1'] = round(v['f1'], 4) * 100

width = 3.281
height = width / 1.618
height = 2.3
fig, ax = plt.subplots(figsize=(width, height))
for i, (k, v) in enumerate(results_f1.items()):
    f1 = v['f1']
    bar = ax.bar(k, f1, color="teal" if i != 1 else 'chocolate', edgecolor="black", zorder=1)
    ax.bar_label(bar, color="darkgreen", zorder=2, backgroundcolor=[1, 1, 1, 0.5], padding=3, bbox=dict(facecolor=[1, 1, 1, 0.75], edgecolor='none', boxstyle='round,pad=0.2'))
    
    fig.text(bar[0].get_x() + bar[0].get_width() / 2, 5, f"Drone", ha='center', va='bottom', color="white", fontsize=15)
    
    if v['use_drones']:
        ax.text(bar[0].get_x() + bar[0].get_width() / 2, 3, r"\textbf{$D_{train}^{drone}$}%", ha='center', va='bottom', color="w", fontsize=font_size)
    if v['use_pl']:
        ax.text(bar[0].get_x() + bar[0].get_width() / 2, 13, r"\textbf{$D_{train}^{pl}$}%", ha='center', va='bottom', color="w", fontsize=font_size)
    if v.get('img', None):
        img = plt.imread(f"results/preds/{v['img']}")
        bar_x = bar[0].get_x()  # Bar's x position
        bar_width = bar[0].get_width()  # Bar's width
        bar_height = f1  # Bar's height (the F1 score)

        # Define absolute position for bbox_to_anchor
        bbox = Bbox.from_bounds(bar_x + bar_width / 2 - 0.1, bar_height * 0.5, 1, 1)

        # Create an inset axes for the image
        ax_inset = inset_axes(ax, width="20%", height="30%", bbox_to_anchor=bbox, bbox_transform=ax.transData, loc='center')

        # Display the image inside the inset
        ax_inset.imshow(img, zorder=4)
        ax_inset.axis('off')




ax.grid(True, axis="y", color="gray", zorder=0, linestyle="--")
ax.set_ylabel(r"$F1$ Score on \hspace{2.6em} (\%)")
ax.set_ylim(0, 55)
ax.set_axisbelow(True)
ax.text(-0.12, 0.64, r"$D_{test}^{drone}$", fontsize=font_size, color="blue", transform=ax.transAxes, ha='center', va='center', rotation=90)

fig.subplots_adjust(top=0.99, bottom=0.18, left=0.12, right=0.99, wspace=0.3)
fig.set_size_inches(width, 0.8 * height)

fig.savefig('results/M2F_F1.pdf')
fig.savefig('results/M2F_F1.png')
